In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
!pip install tldextract
!pip install xgboost lightgbm
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.0 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from utils import run_ML

In [ ]:
data_dir = "URLdatasetX2_1sub5.csv"
df = pd.read_csv(data_dir,index_col=0)


In [ ]:
df.shape, df.head(2)

((2254, 2),
                                           url        type
 0       http://www.crestonwood.com/router.php  legitimate
 1  http://vamoaestudiarmedicina.blogspot.com/  legitimate)

In [ ]:
# smalldata = df.sample(n = 20000, random_state=1)
#smalldata = df.sample(n = 100, random_state=1) # take random 300 samples
smalldata = df


In [ ]:
# get labels of urls
labels = smalldata.iloc[:,-1].values
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)
print(len(labels))
print(labels[90])

2135
0


### Feature extraction

In [ ]:
from utils import extract_features

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Example usage:
url = "http://app.get24h.net"
url_features = extract_features(url)
print(url_features)

{'domain': 'app.get24h.net', 'num_subdomains': 2, 'contains_ip': 0, 'path_length': 0, 'num_path_segments': 1, 'uses_https': 0, 'file_extension': '', 'count_special_characters': 5, 'count_non_alphanumeric_characters': 5, 'TLD': 'net', 'count_obfuscated_characters': 0, 'letter_ratio_in_url': 0.6666666666666666, 'digit_ratio_in_url': 0.09523809523809523, 'count_equals_in_url': 0, 'NoOfAmpersandInURL': 0, 'CharContinuationRate': 0.14285714285714285, 'ratio_obfuscated_characters': 0.0, 'NoOfQMarkInURL': 0}


In [ ]:
# get numerical and catergorical features
phish_url = []
for link in smalldata['url']:
    url_features = extract_features(link)
    phish_url.append(list(url_features.values())[1:])

In [ ]:
phish_url_df = pd.DataFrame(phish_url, columns = list(url_features.keys())[1:])

In [ ]:
phish_url_df.head(2)

,num_subdomains,contains_ip,path_length,num_path_segments,uses_https,file_extension,count_special_characters,count_non_alphanumeric_characters,TLD,count_obfuscated_characters,letter_ratio_in_url,digit_ratio_in_url,count_equals_in_url,NoOfAmpersandInURL,CharContinuationRate,ratio_obfuscated_characters,NoOfQMarkInURL
0,2,0,11,1,0,php,7,7,com,0,0.810811,0.0,0,0,0.135135,0.0,0
1,2,0,1,1,0,,6,6,com,0,0.857143,0.0,0,0,0.047619,0.0,0


In [ ]:
phish_url_df.iloc[:,5] = pd.Categorical(phish_url_df.iloc[:,5]).codes
phish_url_df.iloc[:,8] = pd.Categorical(phish_url_df.iloc[:,8]).codes

In [ ]:
phish_url_df.head(2)

,num_subdomains,contains_ip,path_length,num_path_segments,uses_https,file_extension,count_special_characters,count_non_alphanumeric_characters,TLD,count_obfuscated_characters,letter_ratio_in_url,digit_ratio_in_url,count_equals_in_url,NoOfAmpersandInURL,CharContinuationRate,ratio_obfuscated_characters,NoOfQMarkInURL
0,2,0,11,1,0,34,7,7,33,0,0.810811,0.0,0,0,0.135135,0.0,0
1,2,0,1,1,0,0,6,6,33,0,0.857143,0.0,0,0,0.047619,0.0,0


In [ ]:
# test on URLs features
run_ML(phish_url_df, labels, "URLdatasetX2_1sub10", "manual")

Run:  0 , fold:  0
Train freq:  [1623, 180]
LightGBM, Run:  0 , fold:  1
Train freq:  [1623, 180]
LightGBM, Run:  0 , fold:  2
Train freq:  [1623, 180]
LightGBM, Run:  0 , fold:  3
Train freq:  [1623, 180]
LightGBM, Run:  0 , fold:  4
Train freq:  [1624, 180]
LightGBM, ['LightGBM']
[0.78]


In [ ]:
## test on numerical URLs features
from utils import extract_numerical_features
phish_url = []
for link in smalldata['url']:
    url_features = extract_numerical_features(link)
    phish_url.append(list(url_features.values()))
#run_ML(np.array(phish_url), labels, "malicious_phish", "manual_numerical")

### Extract graph features

In [ ]:
import requests
from bs4 import BeautifulSoup
#from requests.exceptions import ConnectionError
from requests.exceptions import ConnectionError, Timeout, RequestException

In [ ]:
from bs4 import BeautifulSoup
import requests
import numpy as np

# return root and hyperlinks features
def get_graph_features(idx):
    url = smalldata.iloc[idx, 0]
    root_feature = extract_numerical_features(url)  # dict
    hyperlink_data = [list(root_feature.values())]
    try:
        # find all hyperlinks
        reqs = requests.get(url, timeout=10)
        reqs.encoding = 'utf-8'  # Change encoding here if needed
        try:
            soup = BeautifulSoup(reqs.text, 'html.parser')
        except Exception:
            try:
                soup = BeautifulSoup(reqs.text, 'lxml')
            except Exception:
                try:
                    soup = BeautifulSoup(reqs.text, 'html5lib')
                except Exception:
                    # If all parsers fail, return empty hyperlink data
                    hyperlink_data.append(list(np.zeros(15)))
                    return (idx, hyperlink_data)

        urls = []
        count = 0
        for link in soup.find_all('a'):
            weblink = link.get('href')
            if (weblink is not None) and ('http' in weblink):
                urls.append(weblink)
            count += 1
            if count > 50:
                break
        # extract numerical features in from hyperlinks
        if len(urls) > 0:
            for link in urls:
                try:
                    url_features = extract_numerical_features(link)
                    datalinkssss = list(url_features.values())
                except Exception:
                    datalinkssss = list(np.zeros(15))
                hyperlink_data.append(datalinkssss)
        else:
            hyperlink_data.append(list(np.zeros(15)))

    except Exception:
        hyperlink_data.append(list(np.zeros(15)))
        print(url)

    return (idx, hyperlink_data)


In [ ]:
print(get_graph_features(99))

(99, [[2, 0, 1, 1, 0, 6, 6, 0, 0.8285714285714286, 0.0, 0, 0, 0.11428571428571428, 0.0, 0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]])


In [ ]:
import multiprocessing as mp

def get_graph_features_parallel(i):
    return get_graph_features(i)

if __name__ == '__main__':
    n_test_samples = len(smalldata.index)  # how many links we want to test
    with mp.Pool(processes=12) as pool:  # Sử dụng 12 tiến trình
        results = pool.map(get_graph_features_parallel, range(n_test_samples))


http://www.crestonwood.com/router.php
https://novafashion.ru:443/
http://www.gnu.org/software/emacs/manual/html_node/emacs/Lunar-Phases.html
https://www.sosanhgia.com/
http://www.thedigitallandscaping.com/classroom/
https://www.apkandroid.ru/caja-de-herramientas/mx.com.vepormas.cajadeherramientas/downloading.html
http://vufd.hyperphp.com/amzn
https://uns.ac.id/id/
http://www.themitchelli.com/
https://www.elitewriterslab.com/
https://www.thebalancedmamas.com/
http://www.palmeronicorp.com/softsoftwsweet
http://www.kbstitchdesigns.com/i/successful(1).htm
http://www.routeralley.com/guides/nat.pdf
http://www.sagam.sn/images/mailerhome.php
https://nab.support.fortscratchley.org.au/
http://ilovefuzz.com/viewtopic.php?f=149&t=43267&mobile=mobile
http://www.savitari.com/
http://lbcpzonasegurabeta.rf.gd/
http://kbstitchdesigns.com/i/successful(1).htm
http://community-diskussionsforen-ebay-de-t11.22web.org/
https://www.stroymaster.dp.ua/
https://sitelco.net/res/login.php
http://www.taxpark.com/jo

In [ ]:
import pickle
# data_file = "data/raw_graph_features_v2.pickle" # first version
data_file = "URLdatasetX2_1sub5_PyG_Graph.pickle" # first version
#with open(data_file, "wb") as fp:   #Pickling
    #pickle.dump(results, fp)
with open(data_file, "rb") as fp:   # Unpickling
    results = pickle.load(fp)

## PyG

In [ ]:
# # Transfer data object to GPU.
# device = torch.device('cuda')
# data = data.to(device)

In [ ]:
import torch
from torch_geometric.data import Data, Dataset

class GraphClassificationDataset(Dataset):
    def __init__(self, graphs):
        self.graphs = graphs
        # self.labels = labels

    def __len__(self):
        return len(self.graphs)

    def __getitem__(self, idx):
        graph = self.graphs[idx]
        # label = self.labels[idx]
        return graph

In [ ]:
# Assume you have a list of graphs represented as Data objects and a corresponding list of labels
# Only take the url with more than 4 hyperlinks
graphs = []
for i in range(len(results)):
    idx, graph_feature = results[i]
    n_hyperlinks = len(graph_feature)-1
    child_id = [i+1 for i in range(n_hyperlinks)]
    source_id = list(np.zeros(n_hyperlinks).astype(int))
    edge_index = torch.tensor([source_id + child_id,
                               child_id + source_id], dtype=torch.long)
    x = torch.tensor(graph_feature, dtype=torch.float)
    y = torch.tensor([labels[idx]], dtype=torch.int64)
    data = Data(x=x, edge_index=edge_index, y = y)
    if n_hyperlinks > -1:
        graphs.append(data)

# Create a dataset instance
dataset = GraphClassificationDataset(graphs)

In [ ]:
dataset[10]

Data(x=[18, 15], edge_index=[2, 34], y=[1])

In [ ]:
print()
print(f'Dataset: {dataset}:')
print('====================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object.


Dataset: GraphClassificationDataset(2254):
Number of graphs: 2254
Number of features: 15
Number of classes: 2


In [ ]:
count_1 = 0
count_0 = 0
count_2 = 0
for data in dataset:
  label = data.y
  if label == 1:
    count_1 +=1
  if label == 0:
    count_0 +=1
  if label == 2:
    count_2 +=1
print(count_0)
print(count_1)
print(count_2)


2029
225
0


In [ ]:
# dataset = dataset.shuffle()
n_samples = len(dataset)
train_dataset = dataset[:int(0.8*n_samples)]
test_dataset = dataset[int(0.8*n_samples):]

In [ ]:
len(train_dataset), len(test_dataset)

(835, 209)

In [ ]:
from torch_geometric.loader import DataLoader

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

NameError: name 'train_dataset' is not defined

In [ ]:
from torch.nn import Linear, ModuleList
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool



class GCN(torch.nn.Module):
    def __init__(
            self,
            in_channels: int = 15,
            hidden_channels: int = 64,
            out_channels: int = 3,
            pooling_fn: callable = global_mean_pool,
           # device: device = None,
            nb_layers: int = 3,
    ):
        super(GCN, self).__init__()

        self.pooling_fn = pooling_fn
        #self.device = device
        #self.to(device)

        torch.manual_seed(12345)
        self.convs = ModuleList()
        for i in range(nb_layers):
            if i == 0:
                self.convs.append(GCNConv(in_channels, hidden_channels))
            else:
                self.convs.append(GCNConv(hidden_channels, hidden_channels))

        self.lin = Linear(hidden_channels, out_channels)
        self.embeddings = None

    def forward(self, x, edge_index, batch):
        x = x.to(dtype=torch.float32)

        for i, conv in enumerate(self.convs):
            x = conv(x, edge_index)
            if i != len(self.convs) - 1:
                x = x.relu()

        x = self.pooling_fn(x, batch)
        self.embeddings = x

        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)

        return x

    def reset_parameters(self):
        for layer in self.convs:
            layer.reset_parameters()
        self.lin.reset_parameters()


class GCN_2(GCN):
    def __init__(
            self,
            **kwargs,
    ):
        super().__init__(nb_layers=2, **kwargs)


class GCN_3(GCN):
    def __init__(
            self,
            **kwargs,
    ):
        super().__init__(nb_layers=3, **kwargs)

model = GCN_3()
print(model)

GCN_3(
  (convs): ModuleList(
    (0): GCNConv(15, 64)
    (1-2): 2 x GCNConv(64, 64)
  )
  (lin): Linear(in_features=64, out_features=3, bias=True)
)


In [ ]:
from sklearn.metrics import f1_score
import pandas as pd

model = GCN_3()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
         loss = criterion(out, data.y)  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.



def test(loader):
    model.eval()
    correct = 0
    # for data in loader:  # Iterate in batches over the training/test dataset.
    #     out = model(data.x, data.edge_index, data.batch)
    #     pred = out.argmax(dim=1)  # Use the class with highest probability.
    #     correct += int((pred == data.y).sum())  # Check against ground-truth labels.
    # return correct / len(loader.dataset)  # Derive ratio of correct predictions.
    true_labels = []
    pred_labels = []
    for data in loader:  # Iterate in batches over the training/test dataset.
        out = model(data.x, data.edge_index, data.batch)
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        correct += int((pred == data.y).sum())  # Check against ground-truth labels.
        true_labels += data.y.tolist()
        pred_labels += pred.tolist()
        # print(pred_labels)
    return f1_score(true_labels, pred_labels, average = 'macro')

def saveresult(loader,path):
    model.eval()
    correct = 0
    # for data in loader:  # Iterate in batches over the training/test dataset.
    #     out = model(data.x, data.edge_index, data.batch)
    #     pred = out.argmax(dim=1)  # Use the class with highest probability.
    #     correct += int((pred == data.y).sum())  # Check against ground-truth labels.
    # return correct / len(loader.dataset)  # Derive ratio of correct predictions.
    true_labels = []
    pred_labels = []
    y_scores = []
    for data in loader:  # Iterate in batches over the training/test dataset.
        out = model(data.x, data.edge_index, data.batch)
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        correct += int((pred == data.y).sum())  # Check against ground-truth labels.
        true_labels += data.y.tolist()
        pred_labels += pred.tolist()
        probabilities = torch.softmax(out, dim=1)  # Chuyển đổi đầu ra thành xác suất
        y_scores += probabilities[:, 1].tolist()  # Lấy xác suất của lớp dương (lớp 1)
    df = pd.DataFrame({
        'True Label': true_labels,
        'Predicted Label': pred_labels,
        'y_score': y_scores
    })
    df.to_csv(path, index=False)


In [ ]:
from torch_geometric.loader import DataLoader
from sklearn.model_selection import KFold

n_loops = 1
n_folds = 5
base_dir = 'results'
n_samples = len(dataset)
results_df = pd.DataFrame(columns=['fold', 'Train Acc', 'Test Acc'])
for i in range(n_loops):
  cv = KFold(n_splits=n_folds, shuffle=True, random_state = i)
  for fold, (train_idx, test_idx) in enumerate(cv.split(dataset)):
    path_dir = base_dir +'/' +'URLdatasetX2_1sub5'+ '_run_'+str(i)+'_'+ 'fold_'+str(fold)+'_phishGNN_labels.csv'
    print('Run: ', i, ', fold: ', fold)
    train_idx = train_idx.tolist()
    test_idx = test_idx.tolist()
    train_dataset = [dataset[idx] for idx in train_idx]
    test_dataset = [dataset[idx] for idx in test_idx]
    #train_dataset = dataset[train_idx]
    #test_dataset = dataset[test_idx]
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    # Thêm kết quả của epoch vào DataFrame
    results_df.loc[len(results_df)] = [fold, train_acc, test_acc]
    print(f'fold: {fold}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

    # Lấy nhãn thực tế và nhãn dự đoán từ test_loader
    #actual_labels, predicted_labels = get_actual_and_predicted_labels(test_loader)
    saveresult(test_loader,path_dir)
results_df.to_csv("Result.csv", index=False)

Run:  0 , fold:  0
fold: 0, Train Acc: 0.7094, Test Acc: 0.7193
Run:  0 , fold:  1
fold: 1, Train Acc: 0.7643, Test Acc: 0.7534
Run:  0 , fold:  2
fold: 2, Train Acc: 0.7464, Test Acc: 0.7423
Run:  0 , fold:  3
fold: 3, Train Acc: 0.7352, Test Acc: 0.7563
Run:  0 , fold:  4
fold: 4, Train Acc: 0.6880, Test Acc: 0.6652


In [ ]:
print(len(labels))

2135


In [ ]:
#Chạy stratifiedFold
from torch_geometric.loader import DataLoader
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

n_loops = 1
n_folds = 5
base_dir = 'results'
n_samples = len(dataset)
results_df = pd.DataFrame(columns=['fold', 'Train Acc', 'Test Acc'])
for i in range(n_loops):
  cv = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state = i)
  for fold, (train_idx, test_idx) in enumerate(cv.split(dataset,labels)):
    path_dir = base_dir +'/' +'URLdatasetX2_1sub5'+ '_run_'+str(i)+'_'+ 'fold_'+str(fold)+'_phishGNN_labels.csv'
    print('Run: ', i, ', fold: ', fold)
    train_idx = train_idx.tolist()
    test_idx = test_idx.tolist()
    train_dataset = [dataset[idx] for idx in train_idx]
    test_dataset = [dataset[idx] for idx in test_idx]
    #train_dataset = dataset[train_idx]
    #test_dataset = dataset[test_idx]
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    # Thêm kết quả của epoch vào DataFrame
    results_df.loc[len(results_df)] = [fold, train_acc, test_acc]
    print(f'fold: {fold}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

    # Lấy nhãn thực tế và nhãn dự đoán từ test_loader
    #actual_labels, predicted_labels = get_actual_and_predicted_labels(test_loader)
    saveresult(test_loader,path_dir)
results_df.to_csv("Result.csv", index=False)

Run:  0 , fold:  0
fold: 0, Train Acc: 0.4872, Test Acc: 0.4874
Run:  0 , fold:  1
fold: 1, Train Acc: 0.4872, Test Acc: 0.4874
Run:  0 , fold:  2
fold: 2, Train Acc: 0.4872, Test Acc: 0.4874
Run:  0 , fold:  3
fold: 3, Train Acc: 0.4872, Test Acc: 0.4874
Run:  0 , fold:  4
fold: 4, Train Acc: 0.4874, Test Acc: 0.4868
